In [9]:
import numpy as np
import time

In [6]:
# Global constants
SIZE = (2 << 28)
ITERATIONS = 40

In [164]:
def run_numpy_addition(a):
    for i in xrange(ITERATIONS):
        a += i
    print "Result:", a.sum()

In [10]:
a = np.zeros((SIZE), dtype='int32')
run_numpy_addition(a)

Result: 418759311360


In [13]:
# wow thats really slow can we make it better? here comes weld

In [26]:
from weld.weldobject import *
from weld.types import *

In [27]:
# We'll make a Weld enabled NumPy implementation. Add some more text here

In [149]:
class HelloWeldVector(object):
    pass

In [150]:
# Wow some text

In [151]:
def __init__(self, vector):
    self.vector = vector
    self.weldobj = WeldObject(_encoder, _decoder)
    name = self.weldobj.update(vector, WeldVec(WeldInt()))
    self.weldobj.weld_code = name
    self.cached = None

In [152]:
# wao moar text

In [153]:
def add(self, number):
    self.cached = None
    template = "map({0}, |e| e + {1})"
    self.weldobj.weld_code = template.format(
        self.weldobj.weld_code, str(number))
    return self

def __iadd__(self, other):
    return self.add(other)

In [154]:
# wow!

In [155]:
def vector_sum(self):
    self.cached = None
    template = "result(for({0}, merger[i64,+], |b,i,e| merge(b, i64(e))))"
    prev_code = self.weldobj.weld_code
    self.weldobj.weld_code = template.format(self.weldobj.weld_code)
    self.weldobj.decoder = ScalarDecoder()
    result = self.weldobj.evaluate(WeldLong(), verbose=False)
    self.weldobj.decoder = _decoder
    self.weldobj.weld_code = prev_code
    return result

In [156]:
# more text

In [157]:
setattr(HelloWeldVector, '__init__', classmethod(__init__))
setattr(HelloWeldVector, 'add', add)
setattr(HelloWeldVector, '__iadd__', __iadd__)
setattr(HelloWeldVector, 'sum', sum)

In [158]:
# Wait...how do we KNOW HOW TO PASS NUMPY INTO WELD?????
# INEcNODERLKJADS!!!

In [159]:
from weld.encoders import NumpyArrayEncoder, NumpyArrayDecoder, ScalarDecoder

_encoder = NumpyArrayEncoder()
_decoder = NumpyArrayDecoder()

In [160]:
# Now lets try running it again

In [166]:
def run_numpy_addition_with_weld(a):
    a = HelloWeldVector(a)
    for i in xrange(ITERATIONS):
        a += i
        
    print "Result:", a.sum()

In [168]:
def compare_against_numpy():
    a = np.zeros((SIZE), dtype='int32')

    start = time.time()
    run_numpy_addition(a)
    end = time.time()

    numpy_time = end - start

    # Because caches are funny things
    a = np.zeros((SIZE), dtype='int32')

    start = time.time()
    run_numpy_addition_with_weld(a)
    end = time.time()

    run_time = end - start

    print "Speedup:", numpy_time / run_time

In [171]:
import os
os.environ["WELD_NUM_THREADS"] = "1"

compare_against_numpy()

Result: 418759311360
Result: 418759311360
Speedup: 7.4842164901


In [173]:
import os
os.environ["WELD_NUM_THREADS"] = "4"

compare_against_numpy()

Result: 418759311360
Result: 418759311360
Speedup: 9.60425773098


In [ ]:
# Write some more about "real workloads" and Pandas and Grizzly

In [11]:
import pandas as pd
import grizzly.grizzly as gr

In [12]:
def get_data_cleaning_data():
    na_values = ['NO CLUE', 'N/A', '0']
    requests = pd.read_csv('data/311-service-requests.csv', na_values=na_values, dtype={'Incident Zip': str})
    return requests

In [13]:
def run_data_cleaning_pandas(requests):
    requests['Incident Zip'] = requests['Incident Zip'].str.slice(0, 5)
    
    # Fix requests with 00000 zipcodes
    zero_zips = requests['Incident Zip'] == '00000'
    requests['Incident Zip'][zero_zips] = np.nan
    
    # Display unique zip codes.
    print 'Result:', requests['Incident Zip'].unique()

In [14]:
def run_data_cleaning_grizzly(requests):
    requests = gr.DataFrameWeld(requests)
    requests['Incident Zip'] = requests['Incident Zip'].str.slice(0, 5)
    
    # Fix requests with 00000 zipcodes
    zero_zips = requests['Incident Zip'] == '00000'
    requests['Incident Zip'][zero_zips] = 'nan'
    
    # Display unique zip codes.
    print 'Result:', requests['Incident Zip'].unique().evaluate(verbose=False)

In [7]:
requests_orig = get_data_cleaning_data()

In [15]:
requests = requests_orig.copy()

start = time.time()
run_data_cleaning_pandas(requests)
end = time.time()
print "Total time taken: %.3f" % (end - start)

Result: ['11432' '11378' '10032' '10023' '10027' '11372' '11419' '11417' '10011'
 '11225' '11218' '10003' '10029' '10466' '11219' '10025' '10310' '11236'
 '10033' '11216' '10016' '10305' '10312' '10026' '10309' '10036' '11433'
 '11235' '11213' '11379' '11101' '10014' '11231' '11234' '10457' '10459'
 '10465' '11207' '10002' '10034' '11233' '10453' '10456' '10469' '11374'
 '11221' '11421' '11215' '10007' '10019' '11205' '11418' '11369' '11249'
 '10005' '10009' '11211' '11412' '10458' '11229' '10065' '10030' '11222'
 '10024' '10013' '11420' '11365' '10012' '11214' '11212' '10022' '11232'
 '11040' '11226' '10281' '11102' '11208' '10001' '10472' '11414' '11223'
 '10040' '11220' '11373' '11203' '11691' '11356' '10017' '10452' '10280'
 '11217' '10031' '11201' '11358' '10128' '11423' '10039' '10010' '11209'
 '10021' '10037' '11413' '11375' '11238' '10473' '11103' '11354' '11361'
 '11106' '11385' '10463' '10467' '11204' '11237' '11377' '11364' '11434'
 '11435' '11210' '11228' '11368' '11694' '1

In [ ]:
import os
os.environ["WELD_NUM_THREADS"] = "1"

requests = requests_orig.copy()

start = time.time()
run_data_cleaning_grizzly(requests)
end = time.time()
print "Total time taken: %.3f" % (end - start)

In [ ]:
import os
os.environ["WELD_NUM_THREADS"] = "4"

start = time.time()
run_data_cleaning_grizzly(requests)
end = time.time()
print "Total time taken: %.3f" % (end - start)